In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from collections import defaultdict
import hashlib



In [7]:
# Setup driver
driver = webdriver.Chrome()
url = "https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9"
driver.get(url)

WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "select_level_id")))
options = driver.find_elements(By.CSS_SELECTOR, "#select_level_id option")


In [8]:
# Filter Varsity seasons from 2016-17 onward
valid_seasons = []
for i in range(len(options)):
    text = options[i].text.strip()
    value = options[i].get_attribute("value")
    if "Varsity" in text and not any(x in text for x in ["Junior", "C-Team"]):
        if i > 0 and not options[i - 1].get_attribute("value").startswith("?"):
            year = options[i - 1].text.strip()
            if year >= "2016-17":
                valid_seasons.append((year, value))

# Sort by year ascending
def season_sort_key(season_str):
    return int(season_str.split("-")[0])
valid_seasons.sort(key=lambda x: season_sort_key(x[0]))

all_matches = []
all_standings = []
team_class = defaultdict(lambda: "Unknown")
match_counter = 1

def generate_game_id(season, date, time, home_team, away_team):
    to_hash = f"{season}_{date}_{time}_{home_team}_{away_team}"
    return hashlib.md5(to_hash.encode()).hexdigest()

In [9]:
# Scraping loop
for year, value in valid_seasons:
    link = f"https://www.metroleaguewa.org/sport/{value}"
    print(f"Scraping {year} Varsity → {link}")
    driver.get(link)
    time.sleep(3)

    # Set Full Season filter
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "filter_date_range_kword")))
        dropdown = driver.find_element(By.ID, "filter_date_range_kword")
        driver.execute_script(
            "arguments[0].value='season'; arguments[0].dispatchEvent(new Event('change'));", dropdown
        )
        time.sleep(3)
    except Exception as e:
        print(f"Failed to set Full Season filter: {e}")

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "schedule_contents")))
        schedule_blocks = driver.find_elements(By.CLASS_NAME, "schedule_contents")

        for block in schedule_blocks:
            try:
                date_heading = block.find_element(By.CLASS_NAME, "schedule_date_heading").text.strip()
                event_rows = block.find_elements(By.CLASS_NAME, "event_row")

                for row in event_rows:
                    try:
                        time_str = row.find_element(By.CLASS_NAME, "event_time").text.strip()
                        try:
                            game_type = row.find_element(By.CLASS_NAME, "game_type").text.strip()
                        except:
                            game_type = ""

                        if game_type in ["Scrimmage", "Jamboree"]:
                            continue

                        away_team = row.find_element(By.CSS_SELECTOR, ".event_team .event_team_name").text.strip()
                        away_score = row.find_element(By.CSS_SELECTOR, ".event_team .event_team_score").text.strip()
                        home_team = row.find_element(By.CSS_SELECTOR, ".event_team_home .event_team_name").text.strip()
                        home_score = row.find_element(By.CSS_SELECTOR, ".event_team_home .event_team_score").text.strip()

                        match_data = {
                            "Season": year,
                            "Date": date_heading,
                            "Time": time_str,
                            "Home Team": home_team,
                            "Away Team": away_team,
                            "Home Score": home_score,
                            "Away Score": away_score,
                            "Game Type": game_type,
                            "GameID": generate_game_id(year, date_heading, time_str, home_team, away_team),
                            "MatchOrder": match_counter,
                            "Home Classification": team_class[home_team],
                            "Away Classification": team_class[away_team],
                        }
                        all_matches.append(match_data)
                        match_counter += 1

                    except Exception as e:
                        print(f"Failed to parse match row: {e}")

            except Exception as e:
                print(f"Failed to parse schedule block: {e}")

    except Exception as e:
        print(f"Match load error for {year}: {e}")

driver.quit()

# Save raw scraped data
matches_df = pd.DataFrame(all_matches)
standings_df = pd.DataFrame(all_standings)

# Improved date parsing and standardization
def parse_and_standardize_dates(df):
    """
    Parse and standardize date formats from web scraping.
    Handles various date formats that might come from the website.
    """
    if df.empty:
        return df
    
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Convert Date column to datetime with multiple format attempts
    def parse_date_flexible(date_str):
        if pd.isna(date_str) or date_str == '':
            return pd.NaT
        
        # Common date formats that might appear on the website
        date_formats = [
            '%A, %B %d, %Y',      # Monday, January 15, 2024
            '%A, %b %d, %Y',      # Mon, Jan 15, 2024  
            '%B %d, %Y',          # January 15, 2024
            '%b %d, %Y',          # Jan 15, 2024
            '%m/%d/%Y',           # 01/15/2024
            '%m-%d-%Y',           # 01-15-2024
            '%Y-%m-%d',           # 2024-01-15
            '%d/%m/%Y',           # 15/01/2024 (alternative format)
        ]
        
        # Try each format
        for fmt in date_formats:
            try:
                return pd.to_datetime(date_str.strip(), format=fmt)
            except (ValueError, TypeError):
                continue
        
        # If specific formats fail, try pandas' flexible parsing
        try:
            return pd.to_datetime(date_str.strip(), infer_datetime_format=True)
        except (ValueError, TypeError):
            print(f"Warning: Could not parse date '{date_str}'. Setting to NaT.")
            return pd.NaT
    
    # Apply the flexible date parsing
    df['Date_Parsed'] = df['Date'].apply(parse_date_flexible)
    
    # Replace the original Date column with standardized format (YYYY-MM-DD)
    df['Date'] = df['Date_Parsed'].dt.strftime('%Y-%m-%d')
    
    # Keep the original date string for reference if needed
    df['Date_Original'] = df['Date'].copy() if 'Date_Original' not in df.columns else df['Date_Original']
    
    # Drop the temporary column
    df = df.drop('Date_Parsed', axis=1)
    
    # Remove rows with unparseable dates
    invalid_dates = df['Date'].isna() | (df['Date'] == 'NaT')
    if invalid_dates.any():
        print(f"Warning: Removing {invalid_dates.sum()} rows with invalid dates")
        df = df[~invalid_dates]
    
    return df

# Apply date standardization to matches
print("Parsing and standardizing dates...")
matches_df = parse_and_standardize_dates(matches_df)

matches_df.to_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_soccer_results_filtered.csv", index=False)
standings_df.to_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_soccer_standings_with_class.csv", index=False)

Scraping 2016-17 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2016-17&level_id=12#schedule
Scraping 2017-18 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2017-18&level_id=12#schedule
Scraping 2018-19 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2018-19&level_id=12#schedule
Scraping 2019-20 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2019-20&level_id=12#schedule
Scraping 2020-21 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2020-21&level_id=12#schedule
Scraping 2021-22 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2021-22&level_id=12#schedule
Scraping 2022-23 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2022-23&level_id=12#schedule
Scraping 2023-24 Varsity → https://www.metroleaguewa.org/sport/?leagueid=8&sportid=9&school_year=2023-24&level_id=12#s

In [10]:
# -------------------------------
# Define normalization mappings
# -------------------------------
TEAM_NAME_CORRECTIONS = {
    "Lakeside": "Lakeside (Seattle)",
    "Lakeside (Sea)": "Lakeside (Seattle)",
    "Seattle Prep.": "Seattle Prep",
    # Add more corrections here as needed
}

def normalize_team_name(name):
    return TEAM_NAME_CORRECTIONS.get(name, name)

# -------------------------------
# Load and normalize match results
# -------------------------------
matches_df = pd.read_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_soccer_results_filtered.csv")

# Ensure dates are properly parsed when loading from CSV
def ensure_proper_date_format(df):
    """
    Ensure dates are in proper datetime format when loading from CSV.
    This handles cases where dates might be strings or need re-parsing.
    """
    if df.empty or 'Date' not in df.columns:
        return df
    
    df = df.copy()
    
    # Convert Date to datetime if it's not already
    try:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        # Convert back to string format for consistency
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Warning: Issue with date parsing: {e}")
    
    return df

matches_df = ensure_proper_date_format(matches_df)

# Apply normalization to Home and Away Teams
matches_df["Home Team"] = matches_df["Home Team"].apply(normalize_team_name)
matches_df["Away Team"] = matches_df["Away Team"].apply(normalize_team_name)

# -------------------------------
# Load and normalize classification
# -------------------------------
class_df = pd.read_csv(r"C:\Users\User\Desktop\Soccer Footage\school_classification_by_season.csv")

# Normalize classification data too
class_df["School"] = class_df["School"].apply(normalize_team_name)

# -------------------------------
# Merge classification into matches
# -------------------------------
matches_df = matches_df.merge(
    class_df, left_on=["Season", "Home Team"], right_on=["Season", "School"], how="left"
).rename(columns={"Classification": "Home Classification"}).drop(columns=["School"])

matches_df = matches_df.merge(
    class_df, left_on=["Season", "Away Team"], right_on=["Season", "School"], how="left"
).rename(columns={"Classification": "Away Classification"}).drop(columns=["School"])

# -------------------------------
# Clean & Filter Match Data
# -------------------------------
matches_df["Home Score"] = pd.to_numeric(matches_df["Home Score"], errors="coerce")
matches_df["Away Score"] = pd.to_numeric(matches_df["Away Score"], errors="coerce")

# Drop rows with missing scores
matches_df = matches_df.dropna(subset=["Home Score", "Away Score"])

# Improved sorting with proper datetime handling
def sort_matches_by_date(df):
    """
    Sort matches by season and date using proper datetime comparison.
    """
    if df.empty:
        return df
    
    df = df.copy()
    
    # Convert Date to datetime for proper sorting
    df['Date_for_sorting'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Sort by Season and Date (datetime)
    df = df.sort_values(by=["Season", "Date_for_sorting"]).reset_index(drop=True)
    
    # Drop the temporary sorting column
    df = df.drop('Date_for_sorting', axis=1)
    
    return df

# Apply improved sorting
matches_df = sort_matches_by_date(matches_df)


In [11]:
# -------------------------------
# Calculate Standings
# -------------------------------
def calculate_standings(df):
    records = []
    for _, row in df.iterrows():
        hs = int(row["Home Score"])
        as_ = int(row["Away Score"])
        season = row["Season"]
        home = row["Home Team"]
        away = row["Away Team"]

        if hs > as_:
            home_result, away_result = "W", "L"
        elif hs < as_:
            home_result, away_result = "L", "W"
        else:
            home_result = away_result = "T"

        records.extend([
            {"Season": season, "Team": home, "Result": home_result, "GF": hs, "GA": as_},
            {"Season": season, "Team": away, "Result": away_result, "GF": as_, "GA": hs}
        ])

    df = pd.DataFrame(records)
    standings = df.groupby(["Season", "Team"]).agg(
        Wins=("Result", lambda x: (x == "W").sum()),
        Losses=("Result", lambda x: (x == "L").sum()),
        Ties=("Result", lambda x: (x == "T").sum()),
        Goals_For=("GF", "sum"),
        Goals_Against=("GA", "sum")
    ).reset_index()
    standings["Points"] = 3 * standings["Wins"] + 1 * standings["Ties"]
    standings["Games"] = standings["Wins"] + standings["Losses"] + standings["Ties"]
    standings["Win %"] = (standings["Wins"] + 0.5 * standings["Ties"]) / standings["Games"]
    return standings

standings_df = calculate_standings(matches_df)

In [18]:
# -------------------------------
# Filter teams with minimum games played
# -------------------------------
home_games = matches_df.groupby(["Season", "Home Team"]).size().reset_index(name="Home Games")
away_games = matches_df.groupby(["Season", "Away Team"]).size().reset_index(name="Away Games")

home_games.columns = ["Season", "Team", "Home Games"]
away_games.columns = ["Season", "Team", "Away Games"]
games_played = pd.merge(home_games, away_games, on=["Season", "Team"], how="outer").fillna(0)
games_played["Total Games"] = games_played["Home Games"] + games_played["Away Games"]

# Keep only teams with sufficient games
min_games = 3
valid_teams = games_played[games_played["Total Games"] >= min_games][["Season", "Team"]]

matches_df = matches_df.merge(valid_teams, left_on=["Season", "Home Team"], right_on=["Season", "Team"], how="inner").drop(columns=["Team"])
matches_df = matches_df.merge(valid_teams, left_on=["Season", "Away Team"], right_on=["Season", "Team"], how="inner").drop(columns=["Team"])

# -------------------------------
# Run ELO Model
# -------------------------------
def run_elo(matches, base_elo=1500, k=40, hfa=100, cap_margin=3, upset_multiplier=1.5):
    match_counts = defaultdict(int)
    team_elos = defaultdict(lambda: base_elo)
    elo_log = []

    for _, row in matches.iterrows():
        season = row["Season"]
        home = row["Home Team"]
        away = row["Away Team"]
        hs = int(row["Home Score"])
        as_ = int(row["Away Score"])

        # Match result
        result_home = 1 if hs > as_ else 0 if hs < as_ else 0.5

        # Current Elo ratings
        home_elo = team_elos[home]
        away_elo = team_elos[away]

        # Expected result with home field adjustment
        expected_home = 1 / (1 + 10 ** ((away_elo - (home_elo + hfa)) / 400))

        # Margin of victory factor (capped)
        margin = max(1, min(abs(hs - as_), cap_margin))

        # Dynamic K adjustment:
        # For draws, scale reward based on surprise of the result
        if result_home == 0.5:
            surprise_factor = abs(result_home - expected_home)
            k_adjust = 0.5 + (upset_multiplier * surprise_factor)  # dynamic reward
        else:
            k_adjust = 1.0

        # Final Elo change
        change_home = k_adjust * k * margin * (result_home - expected_home)
        change_away = -change_home

        # Update ratings
        team_elos[home] += change_home
        team_elos[away] += change_away

        # Track match count
        match_counts[home] += 1
        match_counts[away] += 1

        # Logging
        elo_log.append({
            "Home Match #": match_counts[home],
            "Away Match #": match_counts[away],
            "Home ELO Change": change_home,
            "Away ELO Change": change_away,
            "Season": season,
            "Date": row["Date"],
            "Home Team": home,
            "Away Team": away,
            "Home Score": hs,
            "Away Score": as_,
            "Home Classification": row.get("Home Classification", "Unknown"),
            "Away Classification": row.get("Away Classification", "Unknown"),
            "Home ELO Before": home_elo,
            "Away ELO Before": away_elo,
            "Home ELO After": team_elos[home],
            "Away ELO After": team_elos[away]
        })

    return pd.DataFrame(elo_log), team_elos

# Run ELO model
matches_df = matches_df.sort_values(by="MatchOrder").reset_index(drop=True)
assert matches_df["MatchOrder"].is_monotonic_increasing, "MatchOrder is not sorted!"
elo_log_df, final_elos = run_elo(matches_df)


In [22]:
# Ensure all output dataframes have consistent date formatting
def ensure_consistent_date_format_for_output(df):
    """
    Ensure consistent date format in output CSV files.
    """
    if df.empty or 'Date' not in df.columns:
        return df
    
    df = df.copy()
    
    # Ensure Date column is in YYYY-MM-DD format
    try:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Warning: Issue formatting dates for output: {e}")
    
    return df

# Apply consistent formatting to all output dataframes
matches_df = ensure_consistent_date_format_for_output(matches_df)
elo_log_df = ensure_consistent_date_format_for_output(elo_log_df)

matches_df.to_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_soccer_results.csv", index=False)
standings_df.to_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_standings_from_results.csv", index=False)
elo_log_df.to_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_elo_log.csv", index=False, float_format="%.2f")
print("Scraping complete. Data saved with improved date handling to CSV files.")


Scraping complete. Data saved to metroleague_soccer_results.csv and metroleague_soccer_standings.csv


In [16]:
# Generate long-format ELO for plotting
home = elo_log_df[["Season", "Home Team", "Home Match #", "Home ELO After"]].rename(
    columns={"Home Team": "Team", "Home Match #": "Match #", "Home ELO After": "ELO"}
)
away = elo_log_df[["Season", "Away Team", "Away Match #", "Away ELO After"]].rename(
    columns={"Away Team": "Team", "Away Match #": "Match #", "Away ELO After": "ELO"}
)
elo_long = pd.concat([home, away], ignore_index=True)

# Remove teams with < 1 games in a season
game_counts = elo_long.groupby(["Season", "Team"]).size().reset_index(name="Games")
valid_teams = game_counts[game_counts["Games"] >= 1][["Season", "Team"]]
elo_long_filtered = elo_long.merge(valid_teams, on=["Season", "Team"])


# Remove teams with fewer than 10 total games across all seasons
total_game_counts = elo_long_filtered.groupby("Team")["Match #"].count().reset_index()
total_game_counts.columns = ["Team", "Total Games"]
valid_teams = total_game_counts[total_game_counts["Total Games"] >= 10]["Team"]
elo_long_cleaned = elo_long_filtered[elo_long_filtered["Team"].isin(valid_teams)]

# Add Match # within each season for each team
elo_long_cleaned["Season Match #"] = elo_long_cleaned.groupby(["Season", "Team"]).cumcount() + 1

elo_long_cleaned.to_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_elo_long.csv", index=False)

print("Scraping + ELO complete. Data saved to CSVs.")

Scraping + ELO complete. Data saved to CSVs.


<ipython-input-16-b1ac94e30a01>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elo_long_cleaned["Season Match #"] = elo_long_cleaned.groupby(["Season", "Team"]).cumcount() + 1


In [17]:
import pandas as pd
import plotly.express as px

# Load your data
df = pd.read_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_elo_long.csv")

# Sort and prep labels
df = df.sort_values(by=["Team", "Season", "Season Match #"])
df["Season Match Label"] = df["Season"] + " | Match " + df["Season Match #"].astype(str)

# Create a match counter across all games for proper plotting
df["Global Match #"] = df.groupby("Team").cumcount() + 1

# Build interactive line plot
fig = px.line(
    df,
    x="Global Match #",
    y="ELO",
    color="Team",
    line_group="Team",
    hover_data=["Team", "Season", "Season Match #", "ELO"],
    title="High School Soccer ELO Progression by Team",
    labels={"Global Match #": "Total Matches Played", "ELO": "ELO Rating"},
)

fig.update_layout(
    hovermode="x unified",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    legend=dict(title="Team", traceorder="normal", itemclick="toggleothers"),
    height=700,
    margin=dict(t=50, b=50),
)

fig.show()


In [343]:
import pandas as pd
import plotly.express as px

# Load long-format ELO
df = pd.read_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_elo_long.csv")

# Sort by team and match number
df = df.sort_values(by=["Team", "Season", "Season Match #"])

# Filter teams with enough total games (optional)
min_games = 10
team_counts = df["Team"].value_counts()
df = df[df["Team"].isin(team_counts[team_counts >= min_games].index)]

# Group by team and season to get first and last ELO
start_elo = df.groupby(["Team", "Season"]).first().reset_index()[["Team", "Season", "ELO"]].rename(columns={"ELO": "Start ELO"})
end_elo = df.groupby(["Team", "Season"]).last().reset_index()[["Team", "Season", "ELO"]].rename(columns={"ELO": "End ELO"})

# Merge into single DataFrame
elo_summary = pd.merge(start_elo, end_elo, on=["Team", "Season"])

# Plot
fig = px.line(
    elo_summary.sort_values(by=["Team", "Season"]),
    x="Season",
    y="End ELO",
    color="Team",
    line_group="Team",
    markers=True,
    hover_data=["Team", "Start ELO", "End ELO"],
    title="📊 Start vs End of Season ELO Progression (All Seasons)",
    labels={"End ELO": "End-of-Season ELO"},
)

# Add start ELO markers
for team in elo_summary['Team'].unique():
    team_data = elo_summary[elo_summary['Team'] == team]
    fig.add_scatter(
        x=team_data['Season'],
        y=team_data['Start ELO'],
        mode='markers',
        marker=dict(symbol='circle-open', size=8),
        name=f"{team} (Start)",
        legendgroup=team,
        showlegend=False,
        hoverinfo='text',
        hovertext=[
            f"{team} — Start ELO: {elo}" for elo in team_data['Start ELO']
        ]
    )

fig.update_layout(
    xaxis_title="Season",
    yaxis_title="ELO Rating",
    height=700,
    hovermode="x unified",
    legend=dict(title="Team", itemclick="toggleothers"),
    margin=dict(t=60, b=80)
)

fig.show()


In [20]:
# Load your ELO match log
elo_df = pd.read_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_elo_log.csv")

# Step 1: Filter 2024–25 matches that have results
test_df = elo_df[
    (elo_df["Season"] == "2024-25") &
    (elo_df["Home Score"].notna()) &
    (elo_df["Away Score"].notna()) &
    ~((elo_df["Home Score"] == 0) & (elo_df["Away Score"] == 0))
].copy()


# Step 2: Calculate Win Probabilities for Home team
def expected_result(home_elo, away_elo, hfa=100):
    return 1 / (1 + 10 ** ((away_elo - home_elo + hfa) / 400))

test_df["Win Prob"] = test_df.apply(
    lambda row: expected_result(row["Home ELO Before"], row["Away ELO Before"]), axis=1
)

# Step 3: Predict result
def classify_prediction(p):
    if p > 0.6:
        return "W"
    elif p < 0.4:
        return "L"
    else:
        return "Toss-up"

test_df["Predicted"] = test_df["Win Prob"].apply(classify_prediction)

# Step 4: Actual result from score
def actual_result(row):
    if row["Home Score"] > row["Away Score"]:
        return "W"
    elif row["Home Score"] < row["Away Score"]:
        return "L"
    else:
        return "T"

test_df["Actual"] = test_df.apply(actual_result, axis=1)

# Step 5: Evaluate
confident_preds = test_df[test_df["Predicted"] != "Toss-up"]
accuracy = (confident_preds["Predicted"] == confident_preds["Actual"]).mean()

# Step 6: Output
print(f"\n📈 ELO Model Accuracy on 2024–25 Season:")
print(f"✅ Confident Predictions (W/L): {len(confident_preds)} matches")
print(f"🎯 Accuracy: {accuracy:.2%}")

# Sample mismatches
mismatches = confident_preds[confident_preds["Predicted"] != confident_preds["Actual"]]
if not mismatches.empty:
    print("\n❌ Sample Mismatches:")
    print(mismatches[[
        "Home Team", "Away Team", "Home Score", "Away Score",
        "Win Prob", "Predicted", "Actual"
    ]].head())
else:
    print("\n✅ No mismatches in confident predictions!")
tossups = test_df[test_df["Predicted"] == "Toss-up"]
tossup_accuracy = (tossups["Actual"] == "T").mean()
print(f"⚖️ Toss-up calls were correct {tossup_accuracy:.2%} of the time (i.e., ended in ties)")



📈 ELO Model Accuracy on 2024–25 Season:
✅ Confident Predictions (W/L): 49 matches
🎯 Accuracy: 57.14%

❌ Sample Mismatches:
        Home Team           Away Team  Home Score  Away Score  Win Prob  \
909  Seattle Prep            Garfield           5           0  0.379405   
913   Nathan Hale     Seattle Academy           1           0  0.300052   
918       Ballard            Garfield           1           1  0.285880   
920      Franklin        West Seattle           3           1  0.279849   
921     Roosevelt  Lakeside (Seattle)           4           3  0.105115   

    Predicted Actual  
909         L      W  
913         L      W  
918         L      T  
920         L      W  
921         L      W  
⚖️ Toss-up calls were correct 11.11% of the time (i.e., ended in ties)


In [21]:
import pandas as pd
import plotly.express as px

# Load the ELO match log
df = pd.read_csv(r"C:\Users\User\Desktop\Soccer Footage\metroleague_elo_log.csv")

# Create a consistent order for all matches per season (scraper's order or raw index)
df["Row Order"] = range(len(df))
df = df.sort_values(by=["Season", "Row Order"])

# Add a Global Match Number per Season (shared league timeline)
df["Global Match #"] = df.groupby("Season").cumcount() + 1

# Assign a row ID to map back after long transformation
df["Match ID"] = df.index

# -----------------------------
# Long-format Transformation
# -----------------------------
home_df = df[[
    "Match ID", "Season", "Global Match #", "Date",
    "Home Team", "Away Team", "Home Score", "Away Score",
    "Home ELO Before", "Home ELO After", "Home ELO Change", "Home Match #"
]].copy()

home_df.columns = [
    "Match ID", "Season", "Global Match #", "Date",
    "Team", "Opponent", "Goals For", "Goals Against",
    "ELO Before", "ELO After", "ELO Δ", "Team Match #"
]
home_df["Location"] = "Home"

away_df = df[[
    "Match ID", "Season", "Global Match #", "Date",
    "Away Team", "Home Team", "Away Score", "Home Score",
    "Away ELO Before", "Away ELO After", "Away ELO Change", "Away Match #"
]].copy()

away_df.columns = [
    "Match ID", "Season", "Global Match #", "Date",
    "Team", "Opponent", "Goals For", "Goals Against",
    "ELO Before", "ELO After", "ELO Δ", "Team Match #"
]
away_df["Location"] = "Away"

# Combine Home and Away
long_df = pd.concat([home_df, away_df], ignore_index=True)

# Compute Match Result (W/L/D)
long_df["Result"] = long_df.apply(
    lambda row: "W" if row["Goals For"] > row["Goals Against"]
    else "L" if row["Goals For"] < row["Goals Against"]
    else "D", axis=1
)

# Clean ELO Δ and match #
long_df["ELO Δ"] = long_df["ELO Δ"].round(2)
long_df["Team Match #"] = long_df["Team Match #"].astype(int)

# -----------------------------
# Filter to Latest Season Only
# -----------------------------
latest_season = long_df["Season"].max()
df_current = long_df[long_df["Season"] == latest_season].copy()

# -----------------------------
# Plot: ELO Progression (Shared League Timeline)
# -----------------------------
fig = px.line(
    df_current,
    x="Global Match #",
    y="ELO After",
    color="Team",
    line_group="Team",
    hover_data={
        "Team": True,
        "Opponent": True,
        "Result": True,
        "Global Match #": True,
        "Team Match #": True,
        "ELO After": True,
        "ELO Δ": True,
        "Location": True,
    },
    title=f"ELO Progression - {latest_season} Season (Shared League Timeline)",
    labels={
        "Global Match #": "League Match Number",
        "ELO After": "ELO Rating",
        "ELO Δ": "ELO Change",
        "Team Match #": "Team's Match #"
    },
)

fig.update_layout(
    hovermode="x unified",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    legend=dict(title="Team", traceorder="normal", itemclick="toggleothers"),
    height=700,
    margin=dict(t=50, b=50),
)

fig.show()

